<a href="https://colab.research.google.com/github/Baek-Donghyeon/Bioinformatics-Algorithms/blob/main/Bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics

In [36]:
import numpy
import random
import re
import sys
from collections import Counter
from collections import defaultdict
import itertools
from functools import reduce
import copy
import heapq
import math
sys.setrecursionlimit(3000)


gen_code = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N','ACA':'T','ACC':'T','ACG':'T','ACU':'T',
            'AGA':'R','AGC':'S','AGG':'R','AGU':'S','AUA':'I','AUC':'I','AUG':'M','AUU':'I',
            'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H','CCA':'P','CCC':'P','CCG':'P','CCU':'P',
            'CGA':'R','CGC':'R','CGG':'R','CGU':'R','CUA':'L','CUC':'L','CUG':'L','CUU':'L',
            'GAA':'E','GAC':'D','GAG':'E','GAU':'D','GCA':'A','GCC':'A','GCG':'A','GCU':'A',
            'GGA':'G','GGC':'G','GGG':'G','GGU':'G','GUA':'V','GUC':'V','GUG':'V','GUU':'V',
            'UAA':'*','UAC':'Y','UAG':'*','UAU':'Y','UCA':'S','UCC':'S','UCG':'S','UCU':'S',
            'UGA':'*','UGC':'C','UGG':'W','UGU':'C','UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

int_mass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'L':113,'N':114,
            'D':115,'K':128,'Q':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

red_mass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'N':114,
            'D':115,'K':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}


def k_mer_of(text, k):

  for i in range(len(text) - k + 1):
    yield text[i:i+k]


def ham(pat, text):

  k = len(pat)
  dis = k

  for k_mer in k_mer_of(text, k):
    dis = min(dis, sum(pat[j] != k_mer[j] for j in range(k)))
  
  return dis


def pat_to_num(pat):

  sym_to_num = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

  if len(pat) == 0:
      return 0
  return 4*pat_to_num(pat[:-1]) + sym_to_num[pat[-1]]


def num_to_pat(num, k):

  num_to_sym = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}

  try:
    if k == 1:
      return num_to_sym(num)
    return num_to_pat(num // 4, k-1) + num_to_sym[num % 4]
  except:
    raise ValueError()


def count_pat(text, pat):

  return sum(k_mer == pat for k_mer in k_mer_of(text, len(pat)))


def find_freq_k_mer(text, k, dis = 0, re = False):

  freq_dict = dict(freq_dict(text, k, dis, re))
  max = max(freq_dict, key = lambda x : freq_dict[x])

  return list(filter(lambda x : x[1] == max, freq_dict.items())) 


def freq_dict(text, k, dis = 0, re = False):
  
  if re:
    return freq_dict(re_complement(text), k, dis) + freq_dict(text, k, dis)
  else:
    return Counter([neigh for k_mer in k_mer_of(text, k) for neigh in neigh(k_mer, dis)])


def re_complement(text):

  compliment_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
  return ''.join(reversed([compliment_dict[base] for base in text]))


def find_pat_occur(text, pat, dis = 0):

  # ham(text, pat) ≤ d
  k = len(pat)
  return [i for i in range(len(text) - k + 1) if ham(text[i:i+k], pat) <= dis]


def find_clump(text, k, L, t):

  '''
  k   length of the k_mer
  L   length of an interval
  t   min ocurrences of a pat in the interval
  '''

  pos_dict = defaultdict(list)
  freq_dict = freq_dict(text, k)
  for pat, freq in freq_dict.items():
    if freq >= t:
      pos_dict[pat] = find_occurrences_of_pat(text, pat)
  for pat, pos in pos_dict.items():
    for i in range(len(pos) - t + 1):
      if pos[i+t-1] - pos[i] <= L - k + 1:
        yield pat


def find_min_skew_pos(text):

  # skew : abs(#'G'-#'C')
  skew = [0]
  for base in text:
    if base == 'C': skew.append(skew[-1]-1)
    elif base == 'G': skew.append(skew[-1]+1)
    else: skew.append(skew[-1])
  min = min(skew)
  return list(filter(lambda x : x == min, skew)) # return minimum skew


def neigh(pat, dis):

  length = len(pat)
  mis_base = itertools.product("ACTG", repeat = dis) # all possible 'dis' length letter strings
  for poss in itertools.combinations(iter(range(length)), dis):
    for base in mis_base:
      neigh = copy.deepcopy(pat)
      for i in dis:
        neigh[pos[i]] = base[i]
        yield neigh
      

def enum_motif(strings, k, dis):

  # find all (k, dis)-motifs in a collect of strings.
  neigh_list = []
  for text in strings:
    neigh_list.append(set(neigh(text, k, dis)))
  motifs = neigh_list[0]
  for neigh in neigh_list:
    motifs = motifs & neigh
  return list(motifs)


def find_median_string(strings, k, dis = 0):

  # a median string for Dna minimizes d(pat, Dna) over all k-mers pat.
  motifs = enum_motif(strings, k, dis)
  if not motifs:
    find_median_string(strings, k, dis + 1)
  else:
    motifs_dict = defaultdict(int)
    for pat in motifs:
      motifs_dict[pat] = sum(ham(pat, text) for text in strings)
    return min(motifs_dict, key = lambda x : motifs_dict[x]) # return key that minimizes the value


def find_prob_k_mer(text, k, profile):

  # A k-mer that was most likely to have been generated by Profile among all k-mers in Text.
  best_score = 0
  for k_mer in k_mer_of(text, k):
    s = dna_score(profile, [k_mer])
    if best_score < s:
      best_score = s
      prob_k_mer = k_mer
  return prob_k_mer


def dna_score(profile, motif):

  str_len = len(motif[0])
  return sum(reduce(lambda x, y: x*y, [profile[pat_to_num(k_mer[i])][i]
              for i in range(str_len)]) for k_mer in motif)


def form_profile(motif):

  k = len(motif[0])
  profile = numpy.full((4,k), 1) # pseudocount
  for i in range(k):
    for k_mer in motif:
      profile[pat_to_num(k_mer[i])][i] += 1
  return profile/(len(motif)+4)


def greedy_motif_search(strings, k):

  best_motif = [text[:k] for text in strings]
  for k_mer in k_mer_of(strings[0], k):
    motif = [k_mer]
    for j in range(1,len(strings)):
      profile = form_profile(motif)
      motif.append(find_prob_k_mer(strings[j], k, profile))
    if dna_score(form_profile(best_motif), best_motif) < dna_score(form_profile(motif), motif):
      best_motif = motif
  return best_motif


def gibbs_sampler(strings, k, n):
  str_len = len(strings[0])
  t = len(strings)
  best_motif_score = 0

  for time in range(n):

    # generate random motif
    motif = []
    for i in range(t):
      motif.append(random.choice([k_mer for k_mer in k_mer_of(strings[i], k)]))

    # update the motif the profile matches best
    last = None
    while motif != last:
      last = motif
      for i in range(t):
        motif[i] = find_prob_k_mer(strings[i], k, form_profile([motif[j] for j in range(t) if j != i]))
    
    motif_score = dna_score(form_profile(motif), motif)

    if best_motif_score < motif_score:
      best_motif_score = motif_score
      best_motif = motif
      print(best_motif_score)

  return best_motif


def rand_motif_search(strings, k):

  str_len = len(strings[0])
  t = len(strings)
  best_motif_score = 0 # score
  
  for time in xrange(800):

    # generate random motif
    motif = []
    for i in range(t):
      motif.append(random.choice([k_mer for k_mer in k_mer_of(strings[i], k)]))
    better_motif = motif

    # converge to a local optimum
    while True:
      profile = form_profile(better_motif)
      motif = []
      for j in range(t):
        motif.append(find_prob_k_mer(strings[j], k, profile))
        motif_score = dna_score(profile, better_motif)
      if motif_score < dna_score(form_profile(motif), motif):
        better_motif = motif
      else:
        break

    if best_motif_score < motif_score:
      best_motif_score = motif_score
      best_motif = better_motif

  return best_motif


def find_string(strings, d = None):

  first = strings[0]

  if d != None: # paired
    half = len(first)//2
    return find_string([front[:half] for front in strings]) + find_string([rear[half:] for rear in strings[-half-d:]])

  return first[:-1] + ''.join([text[-1] for text in strings]) 

def de_brujin_graph(strings):

  graph = defaultdict(list)

  if len(strings[0]) == 2: # paired
    for f,r in strings: # ex) GAGA|TTGA, f = GAGA, r = TTGA
      graph[f[:-1]+r[:-1]].append(f[1:]+r[1:])         
  else: # unpaired
    for k_mer in strings:
      graph[k_mer[:-1]].append(k_mer[1:])
  return graph


def eulerian(graph):

  # determine whether it is an eulerian path or cycle
  graph = copy.deepcopy(graph)
  in_degree = Counter(list(itertools.chain(*graph.values())))
  out_degree = Counter([k for k,v in graph.items() for __ in range(len(v))])
  odd_degree = (out_degree - in_degree).keys()
  if odd_degree:
    stack = list(odd_degree) # start from odd_degree / eulerian path
  else:
    stack = [list(graph.keys())[0]] # start from arbitrary node / eulerian cycle

  # DFS
  visit = []
  while stack:
    try:
      stack.append(graph[stack[-1]].pop())
    except: # backtrack
      visit.append(stack.pop())
  visit.reverse()
  return visit


def reconstruct_string(strings, d = None):

  # d = # of bases b/w front and rear ends
  return find_string(eulerian(de_brujin_graph(strings)),d)


def find_k_universal_circular_string(k):

  # [:-k+1] : for circular_string
  return reconstruct_string(list(map(''.join, product('01', repeat=k))))[:-k+1]


def generate_contig(strings):

  graph = de_brujin_graph(strings)

  # branch : in != 1 & out != 1
  in_degree = Counter(list(itertools.chain(*graph.values())))
  out_degree = Counter([k for k,v in graph.items() for __ in range(len(v))])
  branch = set(node for node in in_degree.keys() if in_degree[node] != 1)
  branch = branch & set(node for values in graph.values() for node in values if out_degree[node] != 1)
  
  # append node until branch
  collect = []
  for node in branch:
    while graph[node]:
      contig = [node, graph[node].pop()]
      while contig[-1] not in branch:
        contig.append(graph[contig[-1]].pop())
      collect.append(find_string(contig))
  return collect


def translate(text):

  aa = []
  for i in range(0,len(text),3):
    codon = text[i:i+3]
    if codon in ['UAA','UAG','UGA']:
      break
    aa.append(gen_code[codon])
  return ''.join(aa)


def dna_to_rna(text):
  return re.sub('T','U',text)


def find_encoding_string(text, pep):
  pep_len = len(pep)
  answer = []
  for i in range(len(text)-3*pep_len+1):
    string = text[i:i+3*pep_len]
    if translate(dna_to_rna(string)) == pep or translate(dna_to_rna(re_complement(string))) == pep:
      answer.append(string)
  return answer


def spectrum(pep, cyclo = False):

  pep_len = len(pep)
  spec = [0]

  if cyclo:
    spec.append(sum(pep))
    circle = pep+pep
    for i in range(pep_len):
      for j in range(1, pep_len):
        spec.append(sum(circle[i:i+j]))
    return sorted(spec)
  else: # linear
    for i in range(pep_len):
      for j in range(i, pep_len):
        spec.append(sum(pep[i:j+1]))
    return sorted(spec)

def aa_to_mass(pep):
  return [int_mass[aa] for aa in pep]

def mass(pep):
  return sum([int_mass[aa] for aa in pep])


def num_of_pep(m):
  d = defaultdict(int)
  aa_mass = red_mass.values()
  d[0] = 1 # single amino acid
  for i in range(1,m+1):
    d[i] = sum([d[i-j] for j in aa_mass if i-j >= 0])
  return d[m]


def pep_score(pep, spec, cyclo = False):
  score = 0
  s = spectrum(pep, cyclo)
  # the scoring function takes into account the multiplicity
  for frag in spec:
    if frag in s:
      s.remove(frag)
      score += 1
  return score


def leaderboard_cyclo_seq(spec, n, m):

  candidate = convolution(spec, n)
  leader_board = candidate
  leader_pep = []
  total_mass = max(spec)

  def trim(leader_board, spec, n):
    # return the top m hiehgest-scoring peptides with ties
    l = []
    for pep in leader_board:
      l.append([pep_score(pep, spec),pep])

    l = sorted(l, key = lambda x : x[0], reverse=True)
    for i in range(n+1, len(l)): # top m elements
      if l[n][0] > l[i][0]:
        leader_board = [v for k,v in l[:i-1]]
        break

    return leader_board

  while True:
    l_b = copy.deepcopy(leader_board)
    for pep in l_b:
      sub_mass = sum(pep)
      if sub_mass == total_mass:
        if pep_score(pep, spec, cyclo=True) > pep_score(leader_pep, spec, cyclo=True):
          leader_pep = pep
      elif sub_mass > total_mass:
        leader_board.remove(pep)

    # break if leader_board is empty
    if not leader_board:
      break
    
    leader_board = trim(leader_board, spec, m)
    # add an a.a. to pep
    leader_board = [pep+aa for pep in leader_board for aa in candidate]

  return leader_pep


def convolution(spec, n):

  convol = defaultdict(int)
  answer = []

  # add differences of masses to convol
  for i in spec:
    for j in spec:
      if i-j > 0:
        convol[i-j] += 1
  
  l = sorted(convol.items(), key = lambda x : x[1], reverse=True)
  for i in range(n+1, len(l)): # top m elements
    if l[n][1] > l[i][1]:
      return [[k] for k,v in l[:i-1] if k in red_mass.values()]
      break

  return [[k] for k,v in l if k in red_mass.values()]


def turn_pike(L, A=[0]):

  L = copy.deepcopy(L)
  
  # remove pairwise differencese b/w points in A[:-1] and A[-1]
  for a in A[:-1]:
    L.remove(a - A[-1])
    L.remove(A[-1] - a)
  L.remove(0)

  if not L:
    return sorted(A)
  else:
    try:
      return turn_pike(L, A+[max(L)])
    except:
      return turn_pike(L, A+[max(A)-max(L)])


f = open('/content/drive/My Drive/Colab Notebooks/input.txt', 'r')
L = f.readline()
L = list(map(int,L.split()))
print(' '.join(list(map(str,turn_pike(L)))))


0 16 26 28 30 40 58 69 72 83 98 102 114 115 124 131 146 148 152 153 163 165 168 176 181 193 201 205 208 226 241 252 269 281 299 306 323 338 351 369 385 391 397 408 424 433 445 459 477 482 483 492 496 502 515 527 537 541 545 546 560 565 576 580 588 591 602 611 622 625 642 650 656 674 688 700 716 729 744 758 767 781 788 802 808 811 818 830 842 850 854 867 882 885 892 898 916 923 934 948 961 973 985 994 1009 1019 1023 1035 1050 1060 1063 1078 1093 1110 1119 1138 1146 1151 1156 1169 1171 1175 1178 1190 1207 1214


In [2]:
a = [0] * 2
a[1] += 1
print(a)

[0, 1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
